In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [ ]:
!kaggle competitions download -c predict-closed-questions-on-stack-overflow

 24% 5.00M/20.8M [00:00<00:00, 46.8MB/s]
100% 20.8M/20.8M [00:00<00:00, 101MB/s] 
 38% 9.00M/23.5M [00:00<00:00, 44.0MB/s]
100% 23.5M/23.5M [00:00<00:00, 78.1MB/s]
 60% 14.0M/23.4M [00:00<00:00, 57.6MB/s]
100% 23.4M/23.4M [00:00<00:00, 91.7MB/s]
 85% 41.0M/48.4M [00:00<00:00, 75.2MB/s]
100% 48.4M/48.4M [00:00<00:00, 87.4MB/s]
  0% 0.00/26.2k [00:00<?, ?B/s]
100% 26.2k/26.2k [00:00<00:00, 25.2MB/s]
 95% 29.0M/30.6M [00:00<00:00, 73.9MB/s]
100% 30.6M/30.6M [00:00<00:00, 87.8MB/s]
 93% 43.0M/46.4M [00:00<00:00, 103MB/s] 
100% 46.4M/46.4M [00:00<00:00, 130MB/s]
 81% 27.0M/33.3M [00:00<00:00, 107MB/s] 
100% 33.3M/33.3M [00:00<00:00, 132MB/s]
  0% 0.00/798k [00:00<?, ?B/s]
100% 798k/798k [00:00<00:00, 108MB/s]
 55% 19.0M/34.3M [00:00<00:00, 57.2MB/s]
100% 34.3M/34.3M [00:00<00:00, 98.5MB/s]
 76% 25.0M/32.9M [00:00<00:00, 37.8MB/s]
100% 32.9M/32.9M [00:00<00:00, 74.2MB/s]
 90% 54.0M/59.8M [00:00<00:00, 110MB/s] 
100% 59.8M/59.8M [00:00<00:00, 135MB/s]
 95% 28.0M/29.4M [00:00<00:00, 49.4MB/s]


In [ ]:
!unzip /content/train-sample.zip

Archive:  /content/train-sample.zip
  inflating: train-sample.csv        


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if filename == 'train-sample.csv':
            train_df = pd.read_csv(os.path.join(dirname, filename)) 

In [ ]:
train_data = train_df.drop(columns=['OwnerUserId','PostId','PostCreationDate','OwnerCreationDate','ReputationAtPostCreation','OwnerUndeletedAnswerCountAtPostTime','PostClosedDate'])

In [ ]:
train_data.head()

,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,OpenStatus
0,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,mongodb,NaN,NaN,NaN,NaN,open
1,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,dom,xsd,jaxp,NaN,NaN,open
2,Too many lookup tables,What are the adverse effects of having too man...,sql-server,database-design,enums,NaN,NaN,open
3,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,php,vb.net,NaN,NaN,NaN,too localized
4,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",mongodb,spring-data,NaN,NaN,NaN,open


In [ ]:
train_data.replace(np.NaN, ' ', inplace=True)

In [ ]:
train_data['Text'] = train_data['Title'] + ' ' + \
                        train_data['BodyMarkdown'] + ' ' + \
                        train_data['Tag1'] + ' ' + \
                        train_data['Tag2'] + ' ' + \
                        train_data['Tag3'] + ' ' + \
                        train_data['Tag4'] + ' ' + \
                        train_data['Tag5']

In [ ]:
train_data.head()

,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,OpenStatus,Text
0,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,mongodb,,,,,open,For Mongodb is it better to reference an objec...
1,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,dom,xsd,jaxp,,,open,How to insert schemalocation in a xml document...
2,Too many lookup tables,What are the adverse effects of having too man...,sql-server,database-design,enums,,,open,Too many lookup tables What are the adverse e...
3,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,php,vb.net,,,,too localized,What is this PHP code in VB.net I am looking f...
4,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",mongodb,spring-data,,,,open,Spring-Data mongodb querying multiple classes ...


In [ ]:
train_data['Text'] = train_data['Text'].str.strip()

In [ ]:
train_data['Output'] = train_data['OpenStatus'].astype('category').cat.codes

In [ ]:
train_data.drop(columns=['Title','BodyMarkdown','Tag1','Tag2','Tag3','Tag4','Tag5','OpenStatus'], inplace=True)

In [ ]:
train_data.head()

,Text,Output
0,For Mongodb is it better to reference an objec...,3
1,How to insert schemalocation in a xml document...,3
2,Too many lookup tables What are the adverse e...,3
3,What is this PHP code in VB.net I am looking f...,4
4,Spring-Data mongodb querying multiple classes ...,3


# Multi-class Classification

1. Preprocessing pipeline

In [ ]:
import torch

inps = train_data['Text'].values
tgts = train_data['Output'].values

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')  # can we try other tokenizers?
train_iter = iter(zip(tgts, inps))

In [ ]:
next(train_iter)

(3,
 'For Mongodb is it better to reference an object or use a natural String key? I am building a corpus of indexed sentences in different languages. I have a collection of Languages which have both an ObjectId and the ISO code as a key. Is it better to use a reference to the Language collection or store a key like "en" or "fr"?\r\n\r\nI suppose it\'s a compromise between:\r\n\r\n - ease of referencing the Language\r\n - object in that collection\r\n - speed in doing queries where the sentence has a certain language\r\n - the size of the data on disk\r\n\r\nAny best practices that I should know of? mongodb')

In [ ]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

train_iter = iter(zip(tgts, inps))
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
vocab(['here', 'is', 'an', 'example'])

[109, 12, 36, 155]

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [ ]:
text_pipeline('here is the an example')

[109, 12, 5, 36, 155]

In [ ]:
label_pipeline('0')

0

In [ ]:
torch.tensor(text_pipeline('here is the an example'), dtype=torch.int64).size(0)

5

**Collate_fn** ✈
With this collate_fn function, you always gonna have a tensor where all your examples have the same size.

In [ ]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))  # returns the number of tokens in an example.
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)  # returns the cumulative sum of elements of input in the dimension dim.
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = iter(zip(tgts, inps))
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch) # what happens if we increase batch size?

## 1. Bag of Embedding Model

In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
train_iter = iter(zip(tgts, inps))
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = iter(zip(tgts, inps))
#train_iter, test_iter = iter(zip(tgts, inps)), iter(zip(tgts_test, inps_test))
train_dataset = to_map_style_dataset(train_iter)
#test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
#test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
#                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 2083 batches | accuracy    0.541
| epoch   1 |  1000/ 2083 batches | accuracy    0.598
| epoch   1 |  1500/ 2083 batches | accuracy    0.608
| epoch   1 |  2000/ 2083 batches | accuracy    0.624
-----------------------------------------------------------
| end of epoch   1 | time: 43.65s | valid accuracy    0.629 
-----------------------------------------------------------
| epoch   2 |   500/ 2083 batches | accuracy    0.634
| epoch   2 |  1000/ 2083 batches | accuracy    0.635
| epoch   2 |  1500/ 2083 batches | accuracy    0.633
| epoch   2 |  2000/ 2083 batches | accuracy    0.638
-----------------------------------------------------------
| end of epoch   2 | time: 43.60s | valid accuracy    0.633 
-----------------------------------------------------------
| epoch   3 |   500/ 2083 batches | accuracy    0.643
| epoch   3 |  1000/ 2083 batches | accuracy    0.644
| epoch   3 |  1500/ 2083 batches | accuracy    0.648
| epoch   3 |  2000/ 2083 batches | accuracy

In [ ]:
# print('Checking the results of test dataset.')
# accu_test = evaluate(test_dataloader)
# print('test accuracy {:8.3f}'.format(accu_test))

## 2. CNN Classification Model

In [ ]:
class CNNClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(CNNClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)